<h1> Loadings </h1>

In [ ]:
# Basic Open AI Prompt and Answer Stuff
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import AzureOpenAI
import openai

# Chains
from langchain import SQLDatabaseChain

# Agents & Tools
from langchain.agents import Tool
from langchain.tools import BaseTool
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from langchain.chains.llm_math.base import LLMMathChain
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

# Utilities
from langchain.utilities import PythonREPL
import pandas as pd
import random
import tabulate as tb
from sqlalchemy.engine import URL
import sqlalchemy
import os

from langchain.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    #QueryCheckerTool,
    QuerySQLDataBaseTool,
)

print(os.getcwd())

# API Credentials

In [2]:
# 'Vishal API' Key
""""""
openai.api_key = "XXXXXXXXXXXXXXXXXXXXXXXXXX"
openai.api_type = "open_ai"
openai.organization = "org-Yma4uJMDlAwruEJU577Bc0NV"
""""""

''

# Boilerplate LLM & SQL

In [3]:
"""
llm = OpenAI(temperature=0, openai_api_key = openai.api_key, openai_organization = openai.organization, engine = "gpt35")
#llm = ChatOpenAI(temperature=0, openai_api_key = openai.api_key, engine = "gpt35")
"""
 
# gpt4 has a longer STM and by extension steerability with context. Better mathematical understanding.
# gpt35 is more constraint and lesser chance of hallucinations - it lacks creativity compared to gpt4.
""""""
llm = ChatOpenAI(temperature=0, openai_api_key = openai.api_key, model = "gpt-3.5-turbo-0613")
#llm = ChatOpenAI(temperature=0, openai_api_key = openai.api_key, model = "gpt-4-0613")

#llm = OpenAI(temperature=0, openai_api_key = openai.api_key, openai_organization = openai.organization, model = "text-davinci-003") #deprecating in end 2023!
#llm = ChatOpenAI(temperature=0, openai_api_key = openai.api_key, model = "gpt-3.5-turbo")
#llm = ChatOpenAI(temperature=0, openai_api_key = openai.api_key, model = "gpt-4")
""""""

"""llm = llm = AzureOpenAI(
    temperature=0, 
    openai_api_key = openai.api_key, 
    deployment_name="gpt35"
    #deployment_name = "ada"
)"""

driver='{SQL Server Native Client 11.0}'#'{ODBC Driver 18 for SQL Server}'
server='localhost\MSSQLSERVEREXP'
database='Gen_AI_Testing'
trusted_connection='yes'

connection_string = f'DRIVER={driver};SERVER={server};'
connection_string += f'DATABASE={database};'
connection_string += f'TRUSTED_CONNECTION={trusted_connection}'

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = sqlalchemy.create_engine(connection_url)
include_tables=['Client Accounts', 'Products', 'Contacts', 'Sales Line Items'] #Remove spaces - to prevent syntax error. E.g. if [Client Account] vs Client Account throws syntax error
sample_rows_in_table_info = 0 # Sample rows confuses the agent and result in hallucinations.
db = SQLDatabase(engine, include_tables=include_tables, sample_rows_in_table_info = sample_rows_in_table_info)

# This is the same method as the above when we dig into the from_uri method. 


Schema Printing Methods:

In [4]:
db.get_table_info_no_throw(db.get_usable_table_names())
print(db.get_table_info_no_throw(db.get_usable_table_names()))
print(db._metadata.sorted_tables)
for t in db._metadata.sorted_tables: print(t.columns)


CREATE TABLE [Client Accounts] (
	[Client Account ID] VARCHAR(64) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Parent ID] VARCHAR(64) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Client Account Name] VARCHAR(1024) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Annual Revenue] REAL NULL, 
	[Number of Employees] INTEGER NULL, 
	[Business Type] VARCHAR(512) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Relationship Type] VARCHAR(512) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Business Sector] VARCHAR(512) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Credit Rating] VARCHAR(64) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Asset Size $] REAL NULL, 
	[Asset Size Cohort] VARCHAR(64) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__Client A__CE7BEBEE71968212] PRIMARY KEY ([Client Account ID])
)


CREATE TABLE [Contacts] (
	[Contact ID] VARCHAR(64) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Client Account ID] VARCHAR(64) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	

# Basic Chain 

In [5]:
db_chain_basic = SQLDatabaseChain.from_llm(llm, db)

In [6]:
print(db_chain_basic.llm_chain.prompt.template)

You are an MS SQL expert. Given an input question, first create a syntactically correct MS SQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the TOP clause as per MS SQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in square brackets ([]) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CAST(GETDATE() as date) function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to r

# Configured Chain - Prompt 0 Configurations

In [7]:
_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.

ALWAYS adhere to the following query rules:
    - DO NOT use table and column identifiers that do not exist.
    - ALWAYS remember to join the data when using column identifiers from other tables.
    - ALWAYS ensure that column identifiers used in SELECT, WHERE, GROUP BY, HAVING and ORDER BY operations exist in the table identifiers used in the FROM operation.
    - ALWAYS use TOP instead of LIMIT when limiting queries.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If you dont know the answer, please say I am not sure of the correct answer. If someone asks for number of clients, take of count Account column. 

Question: {input}"""

PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

db_chain_prompt0 = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)

In [8]:
db_chain_prompt0("For L5 CreditLens, can you give me the total $ amount of sales in 2022?")
#db_chain_prompt0("For CreditLens in 2022 can you give me the total $ amount of sales?")



> Entering new SQLDatabaseChain chain...
For L5 CreditLens, can you give me the total $ amount of sales in 2022?
SQLQuery:SELECT SUM([Sale Amount $]) AS TotalSalesAmount
FROM [Sales Line Items]
JOIN [Products] ON [Sales Line Items].[Product ID] = [Products].[Product ID]
WHERE [Products].[Product (L5)] = 'CreditLens' AND YEAR([Sale Date]) = 2022
SQLResult: [(124826394.43129063,)]
Answer:Total $ amount of sales for L5 CreditLens in 2022 is $124,826,394.43.
> Finished chain.


{'query': 'For L5 CreditLens, can you give me the total $ amount of sales in 2022?',
 'result': 'Total $ amount of sales for L5 CreditLens in 2022 is $124,826,394.43.'}

In [9]:
#db_chain_prompt0("Can you give me distinct number of client in 2022 for L5 CreditLens?")
db_chain_prompt0("For L5 CreditLens can you give me the distinct number of clients in 2022?")



> Entering new SQLDatabaseChain chain...
For L5 CreditLens can you give me the distinct number of clients in 2022?
SQLQuery:SELECT COUNT(DISTINCT [Client Account ID]) AS [Distinct Clients]
FROM [Sales Line Items]
WHERE [Product (L5)] = 'CreditLens' AND YEAR([Sale Date]) = 2022

ProgrammingError: (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid column name 'Product (L5)'. (207) (SQLExecDirectW)")
[SQL: SELECT COUNT(DISTINCT [Client Account ID]) AS [Distinct Clients]
FROM [Sales Line Items]
WHERE [Product (L5)] = 'CreditLens' AND YEAR([Sale Date]) = 2022]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [10]:
#db_chain_prompt0("For all Product (L5) can you give me # Clients for Region Americas")
#db_chain_prompt0("can you give me total # Clients for Region Americas")
db_chain_prompt0("For L3 Regulatiry can you give me # Clients for Region Americas in year 2021")



> Entering new SQLDatabaseChain chain...
For L3 Regulatiry can you give me # Clients for Region Americas in year 2021
SQLQuery:SELECT COUNT(DISTINCT [Client Account ID]) AS [Number of Clients]
FROM [Sales Line Items] sli
JOIN [Products] p ON sli.[Product ID] = p.[Product ID]
WHERE p.[Product Function (L3)] = 'Regulatory'
AND sli.[Region] = 'Americas'
AND YEAR(sli.[Sale Date]) = 2021
SQLResult: [(13,)]
Answer:Number of Clients for L3 Regulatory in Region Americas in year 2021 is 13.
> Finished chain.


{'query': 'For L3 Regulatiry can you give me # Clients for Region Americas in year 2021',
 'result': 'Number of Clients for L3 Regulatory in Region Americas in year 2021 is 13.'}

In [13]:
#db_chain_prompt0("For L3 Regulatory can you give me # Clients for Region Americas in Year 2021")
#db_chain_prompt0("Can you give me # Clients for Region Americas in Year 2021 where L3 is Regulatory")
db_chain_prompt0("how many Regulatory client did we have in 2021 in the Americas")



> Entering new SQLDatabaseChain chain...
how many Regulatory client did we have in 2021 in the Americas
SQLQuery:SELECT COUNT([Client Account ID]) AS [Number of Regulatory Clients]
FROM [Client Accounts]
WHERE [Business Sector] = 'Regulatory'
AND [Client Account ID] IN (
    SELECT [Client Account ID]
    FROM [Sales Line Items]
    WHERE [Region] = 'Americas'
    AND YEAR([Sale Date]) = 2021
)
SQLResult: [(0,)]
Answer:Number of Regulatory Clients: 0
> Finished chain.


{'query': 'how many Regulatory client did we have in 2021 in the Americas',
 'result': 'Number of Regulatory Clients: 0'}

In [19]:
#db_chain_prompt0("For each Product can you give me XSales for year 2022")
db_chain_prompt0("Can you give me Xsale for year 2022 for all the products where L3 is Regulatory")
#db_chain_prompt0("Can you give me total Xsale for year 2022")




> Entering new SQLDatabaseChain chain...
Can you give me Xsale for year 2022 for all the products where L3 is Regulatory
SQLQuery:SELECT [Product Name], SUM([Sale Amount $]) AS TotalSales
FROM [Sales Line Items] sli
JOIN [Products] p ON sli.[Product ID] = p.[Product ID]
WHERE YEAR([Sale Date]) = 2022 AND p.[Product Function (L3)] = 'Regulatory'
GROUP BY [Product Name]
SQLResult: [('Banking Cloud - Regulatory Reports - Pillar 1 - Remuneration Reports', 6912.048828125), ('Banking Cloud - Regulatory Reports - Pillar 1 - TLAC & MREL Reports', 28153.28189086914), ('Banking Cloud - Regulatory Reports - Pillar 1 G-SIB', 9034.0087890625), ('Banking Cloud - Regulatory Reports - STE SREP NPE', 2344.71630859375), ('Banking Cloud Basel 3 Counterparty Credit Risk (SA-CCR EAD, CEM EAD)', 220015.1513671875), ('Banking Cloud Basel 3 Credit Risk RWA (SA and IRB)', 1768241.71875), ('Banking Cloud Basel 3 Credit Risk RWA (SA Only)', 144213.0), ('Banking Cloud Basel 3 Large Exposures', 767575.75), ('Ban

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 6361 tokens. Please reduce the length of the messages.

In [21]:
db_chain_prompt0("For L4 CreditLens can you give me $ Sales for Region Americas in Year 2022")
#db_chain_prompt0("For L3 Regulatory can you give me # Clients and each Client X sales for Region America in Q3 Year 2021")
#db_chain_prompt0("can you give me Growth % between year 2022 vs year 2021 where L3 is Regulatory")



> Entering new SQLDatabaseChain chain...
For L3 Regulatory can you give me # Clients and each Client X sales for Region America in Q3 Year 2021
SQLQuery:SELECT COUNT(DISTINCT CA.[Client Account ID]) AS [Number of Clients], CA.[Client Account Name], SLI.[Sale Amount $]
FROM [Client Accounts] CA
JOIN [Sales Line Items] SLI ON CA.[Client Account ID] = SLI.[Client Account ID]
JOIN [Products] P ON SLI.[Product ID] = P.[Product ID]
WHERE P.[Product Function (L3)] = 'L3 Regulatory'
AND SLI.[Region] = 'America'
AND DATEPART(QUARTER, SLI.[Sale Date]) = 3
AND DATEPART(YEAR, SLI.[Sale Date]) = 2021
GROUP BY CA.[Client Account ID], CA.[Client Account Name], SLI.[Sale Amount $]
SQLResult: []
Answer:I am not sure of the correct answer.
> Finished chain.


{'query': 'For L3 Regulatory can you give me # Clients and each Client X sales for Region America in Q3 Year 2021',
 'result': 'I am not sure of the correct answer.'}

In [39]:
#db_chain_prompt0("For L3 Regulatory can you give me # Clients and for Region America in Year 2021")
#db_chain_prompt0("how many Regulatory client did we have in 2021 in the America")
#db_chain_prompt0("For L3 Regulatory can you give me Growth % between year 2022 vs year 2021")
#db_chain_prompt0("For L3 Regulatory can you give me Growth change in sales between year 2022 vs year 2021")
db_chain_prompt0("can you give me Growth change in sales between year 2022 vs year 2021 where L3 is Regulatory")



> Entering new SQLDatabaseChain chain...
can you give me Growth change in sales between year 2022 vs year 2021 where L3 is Regulatory
SQLQuery:SELECT 
    SUM([Sale Amount $]) AS TotalSales2022,
    (SELECT SUM([Sale Amount $]) FROM [Sales Line Items] WHERE YEAR([Sale Date]) = 2021 AND [Product Function (L3)] = 'Regulatory') AS TotalSales2021
FROM 
    [Sales Line Items]
WHERE 
    YEAR([Sale Date]) = 2022 AND [Product Function (L3)] = 'Regulatory'

ProgrammingError: (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid column name 'Product Function (L3)'. (207) (SQLExecDirectW); [42S22] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid column name 'Product Function (L3)'. (207)")
[SQL: SELECT 
    SUM([Sale Amount $]) AS TotalSales2022,
    (SELECT SUM([Sale Amount $]) FROM [Sales Line Items] WHERE YEAR([Sale Date]) = 2021 AND [Product Function (L3)] = 'Regulatory') AS TotalSales2021
FROM 
    [Sales Line Items]
WHERE 
    YEAR([Sale Date]) = 2022 AND [Product Function (L3)] = 'Regulatory']
(Background on this error at: https://sqlalche.me/e/14/f405)

In [38]:
#db_chain_prompt0("For L3 Regulatory give me top 10 L4 product name")
db_chain_prompt0("Can you give me unique L4 product name where L3 is Regulatory")



> Entering new SQLDatabaseChain chain...
Can you give me unique L4 product name where L3 is Regulatory
SQLQuery:SELECT DISTINCT [Product Family (L4)] 
FROM [Products] 
WHERE [Product Function (L3)] = 'Regulatory'
SQLResult: [('Banking Cloud',), ('RiskAuthority',)]
Answer:Banking Cloud, RiskAuthority
> Finished chain.


{'query': 'Can you give me unique L4 product name where L3 is Regulatory',
 'result': 'Banking Cloud, RiskAuthority'}

In [27]:
#db_chain_prompt0("For Creditlens can you give me $ Sales for Region Americas in Year 2022")
db_chain_prompt0("Can you give me $ Sales for Region Americas in Year 2021 where L4 is CreditLens")



> Entering new SQLDatabaseChain chain...
Can you give me $ Sales for Region Americas in Year 2021 where L4 is CreditLens
SQLQuery:SELECT SUM([Sale Amount $]) AS TotalSales
FROM [Sales Line Items]
JOIN [Products] ON [Sales Line Items].[Product ID] = [Products].[Product ID]
WHERE [Region] = 'Americas' AND YEAR([Sale Date]) = 2021 AND [Product Family (L4)] = 'CreditLens'
SQLResult: [(66383180.13546085,)]
Answer:$66,383,180.14
> Finished chain.


{'query': 'Can you give me $ Sales for Region Americas in Year 2021 where L4 is CreditLens',
 'result': '$66,383,180.14'}

# Configured Chain - Few Shot Prompting V1

In [11]:
PREFIX1 = """Given an input question, first create a syntactically correct {dialect} SQL query to run, then look at the results of the query and return the answer.

Adhere to the following query rules:
    - ALWAYS remember to join the data when using column identifiers from other tables.
    - ALWAYS ensure that column identifiers used in SELECT, WHERE, GROUP BY, HAVING and ORDER BY operations exist in the table identifiers used in the FROM operation.
    - ALWAYS use TOP instead of LIMIT when limiting queries.
    - Always use the following dot notation in SQL queries [table].[column_name]
    - DO NOT query all columns and rows from a table.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:

{table_info}

If you dont know the answer, please say I am not sure of the correct answer."""


EXAMPLES1 = [
    {
        "Question": "For L5 CreditLens, can you give me the total $ amount of sales in 2022?",
        "SQLQuery": 
"""
SELECT 
    SUM([Sale Amount $]) AS TotalSalesAmount
FROM 
    [Sales Line Items]
JOIN 
    [Products] ON [Sales Line Items].[Product ID] = [Products].[Product ID]
WHERE 
    [Products].[Product (L5)] = 'CreditLens' AND YEAR([Sale Date]) = 2022
""",
        "SQLResult": "[(124826394.43129063,)]",
        "Answer":"Total $ amount of sales for L5 CreditLens in 2022 is $124,826,394.43."
    },
    {
        "Question": "For L5 CreditLens can you give me the distinct number of clients in 2022?",
        "SQLQuery": 
"""
SELECT 
    COUNT(DISTINCT [Client Account ID]) AS [Distinct Client Count]
FROM 
    [Sales Line Items]
JOIN 
    [Products] ON [Sales Line Items].[Product ID] = [Products].[Product ID]
WHERE 
    [Products].[Product (L5)] = 'CreditLens'
AND 
    YEAR([Sales Line Items].[Sale Date]) = 2022
""",
        "SQLResult": "[(482,)]",
        "Answer":"Final answer here: There are 482 distinct clients in 2022 for the L5 product CreditLens."
    },
]

SUFFIX1 = """Question: {input}"""

FEW_SHOT_V1 = FewShotPromptTemplate(
    example_prompt = PromptTemplate(input_variables=["Question", "SQLQuery", "SQLResult", "Answer"], template="Question: {Question}\nSQLResult: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"), 
    prefix = PREFIX1,
    examples = EXAMPLES1,
    suffix = SUFFIX1,
    input_variables=["input", "table_info", "dialect"]
)

#print(FEW_SHOT_V1.format(dialect = "MSSQL", table_info = "XXX", input = "What is the day today?"))

db_chain_fewshotv1 = SQLDatabaseChain.from_llm(llm, db, prompt=FEW_SHOT_V1, verbose=True)

In [16]:
#db_chain_fewshotv1("For L3 Regulatory can you give me # Clients for Region Americas in Year 2021")
#db_chain_fewshotv1("Can you give me # Clients for Region Americas in Year 2021 where L3 is Regulatory ")
db_chain_fewshotv1("how many Regulatory client did we have in 2021 in the Americas")



> Entering new SQLDatabaseChain chain...
how many Regulatory client did we have in 2021 in the Americas
SQLQuery:SELECT 
    COUNT(DISTINCT [Client Account ID]) AS [Distinct Client Count]
FROM 
    [Sales Line Items]
JOIN 
    [Products] ON [Sales Line Items].[Product ID] = [Products].[Product ID]
JOIN 
    [Client Accounts] ON [Sales Line Items].[Client Account ID] = [Client Accounts].[Client Account ID]
WHERE 
    [Products].[Product (L5)] = 'Regulatory'
AND 
    [Client Accounts].[Business Sector] = 'Americas'
AND 
    YEAR([Sales Line Items].[Sale Date]) = 2021

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Ambiguous column name 'Client Account ID'. (209) (SQLExecDirectW)")
[SQL: SELECT 
    COUNT(DISTINCT [Client Account ID]) AS [Distinct Client Count]
FROM 
    [Sales Line Items]
JOIN 
    [Products] ON [Sales Line Items].[Product ID] = [Products].[Product ID]
JOIN 
    [Client Accounts] ON [Sales Line Items].[Client Account ID] = [Client Accounts].[Client Account ID]
WHERE 
    [Products].[Product (L5)] = 'Regulatory'
AND 
    [Client Accounts].[Business Sector] = 'Americas'
AND 
    YEAR([Sales Line Items].[Sale Date]) = 2021]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [15]:
PREFIX1 = """Given an input question, first create a syntactically correct {dialect} SQL query to run, then look at the results of the query and return the answer.

Adhere to the following query rules:
    - ALWAYS remember to join the data when using column identifiers from other tables.
    - ALWAYS ensure that column identifiers used in SELECT, WHERE, GROUP BY, HAVING and ORDER BY operations exist in the table identifiers used in the FROM operation.
    - ALWAYS use TOP instead of LIMIT when limiting queries.
    - Always use the following dot notation in SQL queries [table].[column_name]
    - DO NOT query all columns and rows from a table.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:

{table_info}

If you dont know the answer, please say I am not sure of the correct answer."""


EXAMPLES1 = [
    {
        "Question": "how many Regulatory client did we have in 2021 in the Americas?",
        "SQLQuery": 
"""
SELECT COUNT(DISTINCT ca.Client_Account_ID) AS Distinct_Clients
FROM [Client_Accounts] ca
INNER JOIN [Sales_Line_Items] s ON ca.Client_Account_ID = s.Client_Account_ID
INNER JOIN [Products] p ON s.Product_ID = p.Product_ID
WHERE p.Product_Function_L3 = 'Regulatory'
AND YEAR(s.Sale_Date) = 2021
,
        "SQLResult": "[(172,)]",
        "Answer":"The distinct number of clients for L3 regulatory in 2021 is 172."
    },
    {
        "Question": "For L5 CreditLens can you give me the distinct number of clients in 2022?",
        "SQLQuery": 
"""
SELECT 
    COUNT(DISTINCT [Client Account ID]) AS [Distinct Client Count]
FROM 
    [Sales Line Items]
JOIN 
    [Products] ON [Sales Line Items].[Product ID] = [Products].[Product ID]
WHERE 
    [Products].[Product (L5)] = 'CreditLens'
AND 
    YEAR([Sales Line Items].[Sale Date]) = 2022
""",
        "SQLResult": "[(482,)]",
        "Answer":"Final answer here: There are 482 distinct clients in 2022 for the L5 product CreditLens."
    },
]

SUFFIX1 = """Question: {input}"""

FEW_SHOT_V1 = FewShotPromptTemplate(
    example_prompt = PromptTemplate(input_variables=["Question", "SQLQuery", "SQLResult", "Answer"], template="Question: {Question}\nSQLResult: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"), 
    prefix = PREFIX1,
    examples = EXAMPLES1,
    suffix = SUFFIX1,
    input_variables=["input", "table_info", "dialect"]
)

#print(FEW_SHOT_V1.format(dialect = "MSSQL", table_info = "XXX", input = "What is the day today?"))

db_chain_fewshotv1 = SQLDatabaseChain.from_llm(llm, db, prompt=FEW_SHOT_V1, verbose=True)

SyntaxError: '{' was never closed (3245237700.py, line 25)